In [1]:
import os
import re
import pandas as pd
from tqdm import tqdm


In [2]:
# read the eQTLs of ER-positive and ER-negative breast cancers
# the supplementary table 2 and table 3 of PMID: 23374354
table_files = ["/path/to/your/supp_table2.xlsx", "/path/to/your/supp_table3.xlsx"]
df_eqtl = pd.concat(
    [pd.read_excel(file, usecols=["rsID", "Gene"]) for file in table_files],
    axis=0,
    sort=False,
    ignore_index=True,
)
df_eqtl.drop_duplicates(inplace=True)
print(df_eqtl.head())
print(df_eqtl.shape)


         rsID      Gene
0   rs6004673    CRYBB2
1   rs1995969   B3GALTL
2   rs2157990    ZSWIM7
3    rs202370    ZNF334
4  rs10838184  HSD17B12
(6432, 2)


In [3]:
# The reference genome used by that paper is NCBI36
# We need to get the GRCh38 positions of rsIDs from dbSNP:
# https://ftp.ncbi.nlm.nih.gov/snp/organisms/human_9606_b151_GRCh38p7/VCF/All_20180418.vcf.gz
# Here we provide a small file that only contains the involved rsIDs
# please download the "rsID.txt.gz" from our GitHub repo
snp_file = "/path/to/rsID.txt.gz"
df_snp = pd.read_csv(snp_file, delimiter="\t", compression="gzip")
df_eqtl = pd.merge(df_eqtl, df_snp, on="rsID")
print(df_eqtl.head())
print(df_eqtl.shape)


         rsID      Gene chromosome  position
0   rs6004673    CRYBB2      chr22  25489386
1   rs1995969   B3GALTL      chr13  31300307
2   rs2157990    ZSWIM7      chr17  16058679
3    rs202370    ZNF334      chr20  46529811
4  rs10838184  HSD17B12      chr11  43848310
(6421, 4)


In [4]:
# read the GTF given by TCGA, which can be downloaded from:
# https://api.gdc.cancer.gov/data/25aa497c-e615-4cb7-8751-71f744f9691f
# gtf_path: the path of 'gencode.v26.GRCh38.genes.gtf' file given by GTEx
gtf_path = "/path/to/your/gencode.v22.annotation.gtf.gz"
df_gtf = pd.read_csv(
    gtf_path,
    delimiter="\t",
    comment="#",
    header=None,
    usecols=[0, 2, 3, 4, 6, 8],
    compression="gzip",
)
# extract gene names
df_gtf["gene_name"] = [re.search(r'gene_name "(.+?)"', i).group(1) for i in df_gtf[8]]
# extract gene IDs
df_gtf["gene_id"] = [re.search(r'gene_id "(.+?)"', i).group(1) for i in df_gtf[8]]
del df_gtf[8]
df_gtf.columns = [
    "chromosome",
    "type",
    "start",
    "end",
    "strand",
    "gene_name",
    "gene_id",
]
df_gtf = df_gtf[df_gtf["type"].isin(["exon", "gene"])]
# There are some genes with two different IDs
# One of them have been currently (Oct. 2022) deprecated or renamed, so keep only the right one
# The two ensembl IDs (ENSG00000268500 and ENSG00000105501) of SIGLEC5 are both currently valid
# The former is coding gene and the latter is lncRNA, here we retain the former
keep_genes = {
    "NOTCH2NL": "ENSG000002643423.4",
    "CFHR2": "ENSG00000080910.10",
    "TREX1": "ENSG00000213689.8",
    "HYMAI": "ENSG00000276680.1",
    "SYT15": "ENSG00000204176.12",
    "RNASE11": "ENSG00000173464.13",
    "MTHFS": "ENSG00000136371.8",
    "CLN3": "ENSG00000188603.15",
    "NDUFA7": "ENSG00000267855.4",
    "KLK9": "ENSG00000213022.5",
    "SIGLEC5": "ENSG00000268500.4",
}
df_gtf = df_gtf[
    (~df_gtf["gene_name"].isin(keep_genes.keys()))
    | df_gtf["gene_id"].isin(keep_genes.values())
]
print(df_gtf.head())
print(df_gtf.shape)


  chromosome  type  start    end strand gene_name            gene_id
0       chr1  gene  11869  14409      +   DDX11L1  ENSG00000223972.5
2       chr1  exon  11869  12227      +   DDX11L1  ENSG00000223972.5
3       chr1  exon  12613  12721      +   DDX11L1  ENSG00000223972.5
4       chr1  exon  13221  14409      +   DDX11L1  ENSG00000223972.5
6       chr1  exon  12010  12057      +   DDX11L1  ENSG00000223972.5
(1232352, 7)


In [5]:
# calculate the distances of eQTLs to eGenes
df = []
for gene_name, df_temp in tqdm(df_eqtl.groupby("Gene")):
    gene_pos = (
        df_gtf[df_gtf["gene_name"] == gene_name].copy().drop(columns=["gene_name"])
    )
    if len(gene_pos) == 0:
        continue
    exon_pos = gene_pos[gene_pos["type"] == "exon"]
    gene_pos = gene_pos[gene_pos["type"] == "gene"]
    gene_pos = gene_pos.iloc[0, :].to_dict()
    variant_positions = []
    variant_distances = []
    for _, row in df_temp.iterrows():
        if gene_pos["strand"] == "+":
            tss_distance = row["position"] - gene_pos["start"]
            if row["position"] < gene_pos["start"]:
                # upstream
                variant_positions.append("upstream")
                variant_distances.append(gene_pos["start"] - row["position"])
            elif row["position"] > gene_pos["end"]:
                # downstream
                variant_positions.append("downstream")
                variant_distances.append(row["position"] - gene_pos["end"])
            else:
                if (
                    len(
                        exon_pos[
                            (exon_pos["start"] <= row["position"])
                            & (exon_pos["end"] >= row["position"])
                        ]
                    )
                    > 0
                ):
                    # exon
                    variant_positions.append("exon")
                else:
                    # intron
                    variant_positions.append("intron")
                variant_distances.append(row["position"] - gene_pos["start"])
        else:
            tss_distance = row["position"] - gene_pos["end"]
            if row["position"] > gene_pos["end"]:
                # upstream
                variant_positions.append("upstream")
                variant_distances.append(row["position"] - gene_pos["end"])
            elif row["position"] < gene_pos["start"]:
                # downstream
                variant_positions.append("downstream")
                variant_distances.append(gene_pos["start"] - row["position"])
            else:
                if (
                    len(
                        exon_pos[
                            (exon_pos["start"] <= row["position"])
                            & (exon_pos["end"] >= row["position"])
                        ]
                    )
                    > 0
                ):
                    # exon
                    variant_positions.append("exon")
                else:
                    # intron
                    variant_positions.append("intron")
                variant_distances.append(gene_pos["end"] - row["position"])
    df_temp["variant_position"] = variant_positions
    df_temp["variant_distance"] = variant_distances
    df_temp = df_temp.drop(columns=["chromosome", "position"])
    df.append(df_temp.copy())
df = pd.concat(df, axis=0, sort=False, ignore_index=True)
print(df.head())
print(df.shape)


100%|██████████| 1491/1491 [01:28<00:00, 16.90it/s]


         rsID    Gene variant_position  variant_distance
0  rs16863219   AADAC         upstream            631789
1  rs41526349   AADAC         upstream            660638
2   rs2870415   AADAC         upstream            607062
3  rs12106682   AADAC         upstream            643451
4    rs887282  ABCA10       downstream            488447
(6000, 4)


In [6]:
# calculate the lengths of genes
genes = set(df["Gene"])
df_gene_len = df_gtf[df_gtf["type"] == "gene"][["gene_name", "start", "end"]].copy()
df_gene_len = df_gene_len[df_gene_len["gene_name"].isin(genes)]
df_gene_len["gene_length"] = df_gene_len["end"] - df_gene_len["start"] + 1
df_gene_len.drop(columns=["start", "end"], inplace=True)
print(df_gene_len.head())
print(df_gene_len.shape)


      gene_name  gene_length
2371     UBE2J2        19977
7271       GNB1       105833
9211       HES5         1501
10099  ARHGEF16        26688
18335    APITD1        12367
(1380, 2)


In [7]:
# calculate the lengths of exons
df_exon_len = df_gtf[df_gtf["type"] == "exon"][["gene_name", "start", "end"]].copy()
df_exon_len = df_exon_len[df_exon_len["gene_name"].isin(genes)]
gene_names = []
lengths = []
for gene_name, df_temp in tqdm(df_exon_len.groupby("gene_name")):
    pos = set()
    for _, row in df_temp.iterrows():
        pos.update(range(row["start"], row["end"] + 1))
    gene_names.append(gene_name)
    lengths.append(len(pos))
df_exon_len = pd.DataFrame({"gene_name": gene_names, "exon_length": lengths})
print(df_exon_len.head())
print(df_exon_len.shape)


100%|██████████| 1380/1380 [00:03<00:00, 429.39it/s]

  gene_name  exon_length
0     AADAC         1668
1    ABCA10         8405
2     ABCC1         7575
3    ABCC11         7245
4     ABCC4         7819
(1380, 2)


In [8]:
# the density of eQTLs on exons
# calculate the count of eQTLs on exons
df_gene = df[df["variant_position"].isin(["exon", "intron"])]
df_exon = df_gene[df_gene["variant_position"] == "exon"]
df_nvariants = df_exon[["Gene", "rsID"]].groupby("Gene").count()
# The gene without any eQTLs on its exons will have a zero count
genes_novar = sorted(genes - set(df_nvariants.index))
df_nvariants = pd.concat(
    [
        df_nvariants,
        pd.DataFrame({"rsID": [0] * len(genes_novar)}, index=genes_novar),
    ],
    axis=0,
    sort=False,
    ignore_index=False,
)
# density = total count / total length
df_nvariants = pd.merge(
    df_nvariants, df_exon_len, left_index=True, right_on="gene_name"
)
density_exon = df_nvariants["rsID"].sum() / df_nvariants["exon_length"].sum()
density_exon


2.7843131795464354e-05

In [9]:
# the density of eQTLs on introns
# calculate the count of eQTLs on introns
df_intron = df_gene[df_gene["variant_position"] == "intron"]
df_nvariants = df_intron[["Gene", "rsID"]].groupby("Gene").count()
# The gene without any eQTLs ont its introns will have a zero count
genes_novar = sorted(genes - set(df_nvariants.index))
df_nvariants = pd.concat(
    [
        df_nvariants,
        pd.DataFrame({"rsID": [0] * len(genes_novar)}, index=genes_novar),
    ],
    axis=0,
    sort=False,
    ignore_index=False,
)
# density = total count / total length
df_nvariants = pd.merge(
    df_nvariants, df_gene_len, left_index=True, right_on="gene_name"
)
df_nvariants = pd.merge(df_nvariants, df_exon_len, on="gene_name")
df_nvariants["intron_length"] = (
    df_nvariants["gene_length"] - df_nvariants["exon_length"]
)
density_intron = df_nvariants["rsID"].sum() / df_nvariants["intron_length"].sum()
density_intron


1.0579285196094095e-05

In [10]:
# density of eQTLs in median distance up/downstream
# formula: The number of all up/downstream eQTLs * 0.5 / ((Dup + Ddown) * number of genes)
df_nogene = df[df["variant_position"].isin(["upstream", "downstream"])]
# The number of up/downstream eQTLs
nvariants = len(df_nogene)
# median distances up/downstream
dist_up = (
    df_nogene[df_nogene["variant_position"] == "upstream"]["variant_distance"]
    .median()
    .__ceil__()
)
dist_down = (
    df_nogene[df_nogene["variant_position"] == "downstream"]["variant_distance"]
    .median()
    .__ceil__()
)
dist_up, dist_down
# number of genes
ngenes = len(genes)
density_up_down_stream = nvariants * 0.5 / (dist_up + dist_down) / ngenes
density_up_down_streamdf_nogene = df[
    df["variant_position"].isin(["upstream", "downstream"])
]
# The number of up/downstream eQTLs
nvariants = len(df_nogene)
# median distances up/downstream
dist_up = (
    df_nogene[df_nogene["variant_position"] == "upstream"]["variant_distance"]
    .median()
    .__ceil__()
)
dist_down = (
    df_nogene[df_nogene["variant_position"] == "downstream"]["variant_distance"]
    .median()
    .__ceil__()
)
print(dist_up, dist_down)
# number of genes
ngenes = len(genes)
density_up_down_stream = nvariants * 0.5 / (dist_up + dist_down) / ngenes
print(density_up_down_stream)


129110 84394
7.889408581830189e-06


In [11]:
# ratio of densities
print(density_exon / density_up_down_stream)
print(density_intron / density_up_down_stream)


3.5291785824844797
1.340947814574956
